In [4]:
import os
import zipfile
import numpy as np
import pandas as pd

# Function to extract zip files
def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Function to read .py files from the given directory
def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file skipped: {file_path}")
    return dataPointsList

# Paths to your zip files (you can modify these paths)
flakyZip = "compressedDataset/flaky_files.zip"
nonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

# Create directories for extraction
extractDir = "extracted_files"
flakyDir = os.path.join(extractDir, 'flaky')
nonFlakyDir = os.path.join(extractDir, 'nonFlaky')

# Ensure directories exist
os.makedirs(flakyDir, exist_ok=True)
os.makedirs(nonFlakyDir, exist_ok=True)

# Extract the zip files
extract_zip(flakyZip, flakyDir)
extract_zip(nonFlakyZip, nonFlakyDir)

# Collect data points from flaky and non-flaky files
dataPointsFlaky = getDataPoints(flakyDir)
dataPointsNonFlaky = getDataPoints(nonFlakyDir)
dataPoints = dataPointsFlaky + dataPointsNonFlaky

# Create labels: 1 for flaky, 0 for non-flaky
dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))

print(f"Number of flaky files: {len(dataPointsFlaky)}")
print(f"Number of non-flaky files: {len(dataPointsNonFlaky)}")


Number of flaky files: 45
Number of non-flaky files: 243


KNN

In [15]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from itertools import product

# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

# Summarize and find best parameter combinations
def summarize_and_find_best(df_results):
    summary_df = df_results.groupby(['pca__n_components', 'n_neighbors', 'weights', 'metric', 'threshold']).agg({
        'accuracy': 'mean',
        'precision': 'mean',
        'recall': 'mean',
        'f1': 'mean',
        'mcc': 'mean'
    }).reset_index()

    # Finding the best parameter set based on the highest F1 score
    best_row = summary_df.loc[summary_df['f1'].idxmax()]

    return summary_df, best_row

# Combined SMOTE and Threshold-based KNN function with vectorization in pipeline
def runKNNWithSMOTEAndThreshold(dataPoints, y, outDir, n_splits, param_grid, thresholds):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics_per_combination = []

    for params in product(*param_grid.values()):
        param_dict = dict(zip(param_grid.keys(), params))

        # Define pipeline with CountVectorizer, SMOTE, PCA, and KNN
        pipeline = ImbPipeline([
            ('vectorizer', CountVectorizer(stop_words=None)),
            ('pca', PCA(n_components=param_dict['pca__n_components'], random_state=42)),
            ('smote', SMOTE(random_state=42)),
            ('knn', KNeighborsClassifier(
                n_neighbors=param_dict['n_neighbors'],
                weights=param_dict['weights'],
                metric=param_dict['metric'],
                n_jobs=-1
            ))
        ])

        for fold, (train_index, test_index) in enumerate(skf.split(dataPoints, y)):
            X_train = [dataPoints[i] for i in train_index]
            X_test = [dataPoints[i] for i in test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Train the pipeline
            pipeline.fit(X_train, y_train)

            # Predict probabilities
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

            for threshold in thresholds:
                y_pred_threshold = (y_pred_proba >= threshold).astype(int)

                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred_threshold)
                precision = precision_score(y_test, y_pred_threshold, zero_division=1)
                recall = recall_score(y_test, y_pred_threshold, zero_division=1)
                f1 = f1_score(y_test, y_pred_threshold, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred_threshold)

                # Store results
                metrics_per_combination.append({
                    'pca__n_components': param_dict['pca__n_components'],
                    'n_neighbors': param_dict['n_neighbors'],
                    'weights': param_dict['weights'],
                    'metric': param_dict['metric'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    # Convert metrics to DataFrame and save results
    df_results = pd.DataFrame(metrics_per_combination)
    os.makedirs(outDir, exist_ok=True)
    outFile = os.path.join(outDir, "knn-smote-threshold-results.csv")
    df_results.to_csv(outFile, index=False)

    # Summarize results and return the best parameter set
    summary_df, best_params = summarize_and_find_best(df_results)

    # Save summarized results to a new CSV file
    summary_outFile = os.path.join(outDir, "knn-smote-threshold-summary.csv")
    summary_df.to_csv(summary_outFile, index=False)
    print(f"Summary of results saved to {summary_outFile}")

    return df_results, best_params

# Hyperparameters and thresholds
param_grid = {
    'pca__n_components': [180, 200, 220],
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['cosine', 'euclidean']
}

thresholds = np.linspace(0.1, 0.9, 9)  # Thresholds from 0.1 to 0.9

outDir = "results/"
df_results, best_params = runKNNWithSMOTEAndThreshold(dataPoints, dataLabelsList, outDir, n_splits=5, param_grid=param_grid, thresholds=thresholds)

print("\nKNN with SMOTE and Threshold analysis completed.")
print("Best parameter set based on F1-score:")
print(best_params)


Summary of results saved to results/knn-smote-threshold-summary.csv

KNN with SMOTE and Threshold analysis completed.
Best parameter set based on F1-score:
pca__n_components         180
n_neighbors                11
weights              distance
metric                 cosine
threshold                 0.7
accuracy             0.916045
precision                0.84
recall               0.616667
f1                   0.685583
mcc                  0.664656
Name: 150, dtype: object


SVM

In [10]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from itertools import product
import os

# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

# Summarize and find best parameter combinations
def summarize_and_find_best(df_results):
    summary_df = df_results.groupby(['pca__n_components', 'kernel', 'C', 'threshold']).agg({
        'accuracy': 'mean',
        'precision': 'mean',
        'recall': 'mean',
        'f1': 'mean',
        'mcc': 'mean'
    }).reset_index()

    # Finding the best parameter set based on the highest F1 score
    best_row = summary_df.loc[summary_df['f1'].idxmax()]

    return summary_df, best_row

# Combined SMOTE and Threshold-based SVM with vectorizer
def runSVMWithSMOTEAndThreshold(dataPoints, y, outDir, n_splits, param_grid, thresholds):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics_per_combination = []

    for params in product(*param_grid.values()):
        param_dict = dict(zip(param_grid.keys(), params))

        # Define pipeline with Vectorizer, SMOTE, PCA, and SVM
        pipeline = ImbPipeline([
            ('vectorizer', CountVectorizer(stop_words=None)),
            ('pca', PCA(n_components=param_dict['pca__n_components'], random_state=42)),
            ('smote', SMOTE(random_state=42)),
            ('svm', SVC(
                kernel=param_dict['kernel'],
                C=param_dict['C'],
                probability=True,  # Enable probability predictions for threshold adjustment
                random_state=42
            ))
        ])

        for fold, (train_index, test_index) in enumerate(skf.split(dataPoints, y)):
            X_train = [dataPoints[i] for i in train_index]
            X_test = [dataPoints[i] for i in test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Train the pipeline
            pipeline.fit(X_train, y_train)

            # Predict probabilities
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # SVM probability for positive class

            for threshold in thresholds:
                y_pred_threshold = (y_pred_proba >= threshold).astype(int)

                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred_threshold)
                precision = precision_score(y_test, y_pred_threshold, zero_division=1)
                recall = recall_score(y_test, y_pred_threshold, zero_division=1)
                f1 = f1_score(y_test, y_pred_threshold, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred_threshold)

                # Store results
                metrics_per_combination.append({
                    'pca__n_components': param_dict['pca__n_components'],
                    'kernel': param_dict['kernel'],
                    'C': param_dict['C'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    # Convert metrics to DataFrame and save results
    df_results = pd.DataFrame(metrics_per_combination)
    os.makedirs(outDir, exist_ok=True)
    outFile = os.path.join(outDir, "svm-smote-threshold-results.csv")
    df_results.to_csv(outFile, index=False)

    # Summarize results and return the best parameter set
    summary_df, best_params = summarize_and_find_best(df_results)

    # Save summarized results to a new CSV file
    summary_outFile = os.path.join(outDir, "svm-smote-threshold-summary.csv")
    summary_df.to_csv(summary_outFile, index=False)
    print(f"Summary of results saved to {summary_outFile}")

    return df_results, best_params

# Hyperparameters and thresholds
param_grid = {
    'pca__n_components': [180, 200, 220],              # Number of PCA components to use
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],    # SVM kernel types
    'C': [0.01, 0.1, 1.0, 10.0, 100.0]                 # Regularization parameter C
}

thresholds = np.linspace(0.1, 0.9, 9)  # Thresholds from 0.1 to 0.9

outDir = "results/"
df_results, best_params = runSVMWithSMOTEAndThreshold(dataPoints, dataLabelsList, outDir, n_splits=5, param_grid=param_grid, thresholds=thresholds)

print("\nSVM with SMOTE and Threshold analysis completed.")
print("Best parameter set based on F1-score:")
print(best_params)


Summary of results saved to results/svm-smote-threshold-summary.csv

SVM with SMOTE and Threshold analysis completed.
Best parameter set based on F1-score:
pca__n_components         180
kernel                    rbf
C                        10.0
threshold                 0.3
accuracy             0.925989
precision            0.813853
recall               0.705556
f1                   0.732726
mcc                  0.707262
Name: 119, dtype: object


XGB

In [14]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from itertools import product
import os

# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

# Summarize and find best parameter combinations
def summarize_and_find_best(df_results):
    summary_df = df_results.groupby(['pca__n_components', 'max_depth', 'learning_rate', 'n_estimators', 'threshold']).agg({
        'accuracy': 'mean',
        'precision': 'mean',
        'recall': 'mean',
        'f1': 'mean',
        'mcc': 'mean'
    }).reset_index()

    # Finding the best parameter set based on the highest F1 score
    best_row = summary_df.loc[summary_df['f1'].idxmax()]

    return summary_df, best_row

# Combined SMOTE and Threshold-based XGB with Vectorizer
def runXGBWithSMOTEAndThreshold(dataPoints, y, outDir, n_splits, param_grid, thresholds):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics_per_combination = []

    for params in product(*param_grid.values()):
        param_dict = dict(zip(param_grid.keys(), params))

        # Define pipeline with Vectorizer, SMOTE, PCA, and XGBoost
        pipeline = ImbPipeline([
            ('vectorizer', CountVectorizer(stop_words=None)),  # Vectorization step
            ('pca', PCA(n_components=param_dict['pca__n_components'], random_state=42)),
            ('smote', SMOTE(random_state=42)),
            ('xgb', xgb.XGBClassifier(
                max_depth=param_dict['max_depth'],
                learning_rate=param_dict['learning_rate'],
                n_estimators=param_dict['n_estimators'],
                eval_metric='logloss',  # Set eval metric to avoid warnings
                random_state=42
            ))
        ])

        for fold, (train_index, test_index) in enumerate(skf.split(dataPoints, y)):
            X_train = [dataPoints[i] for i in train_index]
            X_test = [dataPoints[i] for i in test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Train the pipeline
            pipeline.fit(X_train, y_train)

            # Predict probabilities
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # XGBoost probability for positive class

            for threshold in thresholds:
                y_pred_threshold = (y_pred_proba >= threshold).astype(int)

                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred_threshold)
                precision = precision_score(y_test, y_pred_threshold, zero_division=1)
                recall = recall_score(y_test, y_pred_threshold, zero_division=1)
                f1 = f1_score(y_test, y_pred_threshold, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred_threshold)

                # Store results
                metrics_per_combination.append({
                    'pca__n_components': param_dict['pca__n_components'],
                    'max_depth': param_dict['max_depth'],
                    'learning_rate': param_dict['learning_rate'],
                    'n_estimators': param_dict['n_estimators'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    # Convert metrics to DataFrame and save results
    df_results = pd.DataFrame(metrics_per_combination)
    os.makedirs(outDir, exist_ok=True)
    outFile = os.path.join(outDir, "xgb-smote-threshold-results.csv")
    df_results.to_csv(outFile, index=False)

    # Summarize results and return the best parameter set
    summary_df, best_params = summarize_and_find_best(df_results)

    # Save summarized results to a new CSV file
    summary_outFile = os.path.join(outDir, "xgb-smote-threshold-summary.csv")
    summary_df.to_csv(summary_outFile, index=False)
    print(f"Summary of results saved to {summary_outFile}")

    return df_results, best_params

# Hyperparameters and thresholds
param_grid = {
    'pca__n_components': [180, 200, 220],        # PCA components to use
    'max_depth': [3, 5, 7, 10],                  # Max depth of trees
    'learning_rate': [0.01, 0.1, 0.3, 0.5],      # Learning rate for boosting
    'n_estimators': [100, 150, 200]               # Number of boosting rounds
}

thresholds = np.linspace(0.1, 0.9, 9)  # Thresholds from 0.1 to 0.9

outDir = "results/"
df_results, best_params = runXGBWithSMOTEAndThreshold(dataPoints, dataLabelsList, outDir, n_splits=5, param_grid=param_grid, thresholds=thresholds)

print("\nXGBoost with SMOTE and Threshold analysis completed.")
print("Best parameter set based on F1-score:")
print(best_params)


Summary of results saved to results/xgb-smote-threshold-summary.csv

XGBoost with SMOTE and Threshold analysis completed.
Best parameter set based on F1-score:
pca__n_components    180.000000
max_depth              3.000000
learning_rate          0.500000
n_estimators         100.000000
threshold              0.500000
accuracy               0.926271
precision              0.841270
recall                 0.638889
f1                     0.713632
mcc                    0.688261
Name: 85, dtype: float64
